<a href="https://colab.research.google.com/github/Jun0S2/AIBootcamp/blob/main/%EB%85%BC%EB%AC%B8%EC%9A%94%EC%95%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain chromadb openai tiktoken pypdf langchain-community langchain-chroma langchain-openai bs4 requests

In [13]:
!pip show langchain
!pip install langchain-text-splitters

## 설치 목록

* langchain: LangChain의 기본 라이브러리
* langchain-community: PyPDFLoader 같은 커뮤니티 모듈을 제공
* langchain-chroma: ChromaDB를 LangChain과 함께 사용하는 데 필요한 라이브러리
* langchain-openai: OpenAI와 LangChain을 통합하기 위한 패키지
* bs4: BeautifulSoup4를 사용해 HTML 파싱을 수행
* requests: PDF를 다운로드하는 데 필요

In [5]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.schema import Document
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [6]:
def summarize_paper(pdf_url, prompt):
    """
    Summarizes a research paper given its PDF URL and a prompt.

    Args:
        pdf_url (str): URL of the research paper in PDF format.
        prompt (str): Prompt to guide the LLM for summarization.

    Returns:
        str: Summarization result from the LLM.
    """
    # Step 1: Download the PDF
    pdf_path = "paper.pdf"
    response = requests.get(pdf_url)
    with open(pdf_path, "wb") as f:
        f.write(response.content)

    # Step 2: Load the PDF as documents
    loader = PyPDFLoader(pdf_path)
    raw_docs = loader.load()

    # Step 3: Split the documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    split_docs = text_splitter.split_documents(raw_docs)

    # Step 4: Store the chunks in a vectorstore
    vectorstore = Chroma.from_documents(
        documents=split_docs,
        embedding=OpenAIEmbeddings(api_key="...")
    )

    # Step 5: Use retriever to find relevant chunks
    retriever = vectorstore.as_retriever()
    retrieved_docs = retriever.invoke(prompt)

    # Step 6: Format the retrieved documents
    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    context = format_docs(retrieved_docs)

    # Step 7: Use LLM to summarize
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        api_key="..."
    )
    llm_prompt = f"{prompt}\n\nContext:\n{context}"
    response = llm.invoke(llm_prompt)

    # Clean up downloaded PDF
    os.remove(pdf_path)

    return response.content


In [8]:
import requests

In [9]:
# Example usage
pdf_url = "https://arxiv.org/pdf/2005.11401"
prompt = "논문의 주요 연구 목표, 방법론, 그리고 결론을 요약해줘."
summary = summarize_paper(pdf_url, prompt)
print("Summary:\n", summary)

Summary:
 해당 논문 "Coarse-to-fine question answering for long documents"의 주요 연구 목표, 방법론, 결론을 요약하면 다음과 같습니다.

### 주요 연구 목표
이 연구의 주요 목표는 긴 문서에서 효과적으로 질문에 대한 답변을 찾는 방법을 제안하는 것입니다. 특히, 기존의 QA (Question Answering) 시스템들이 긴 문서에서 신뢰할 수 있는 정보에 접근하는 데 어려움을 겪는 문제를 해결하고자 했습니다.

### 방법론
1. **Coarse-to-Fine Strategy**: 연구진은 질문에 대한 답변을 찾기 위해 두 단계의 접근 방식을 사용했습니다. 첫 번째 단계는 문서에서 질문과 관련된 중요한 정보 조각을 식별하는 'Coarse' 단계이며, 두 번째 단계는 이 정보를 바탕으로 더 정교한 'Fine' 단계를 통해 최종 답변을 생성하는 방식입니다.
  
2. **Retrieval-Augmented Generation (RAG)**: 이 시스템은 정보 검색을 통합하여 모델이 이전에 학습한 내용 외에도 새로운 정보를 활용할 수 있게 합니다.

3. **Baseline Comparisons**: 연구진은 제안한 방법을 기존의 state-of-the-art 시스템과 비교하여 성능을 평가했습니다.

### 결론
연구 결과, 제안된 coarse-to-fine 접근 방식이 긴 문서 내에서의 질문 응답 문제를 효과적으로 해결할 수 있음을 보여주었습니다. 해당 방법은 기존의 복잡한 파이프라인 시스템과 비교하여 단순하고 효율적이면서도 경쟁력 있는 성능을 발휘하여, 긴 문서에서의 QA 작업에서 유망한 가능성을 나타냈습니다. 연구 결과는 복잡한 처리 과정을 줄이면서도 높은 정확도를 유지할 수 있는 새로운 접근 방식의 필요성을 강조합니다.

이 요약은 논문의 주요 목표, 방법론 및 결론을 간략하게 정리한 것입니다. 추가적인 상세 내용은 논문의 원문을 참조하시기 바랍니다.
